In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")

/Users/likhit/Documents/Coding/NLP/NLP_basics/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)
model.to(device)
model

mps


Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps=1e-06)

In [3]:
prompt = "What is Machine Learning is the best that humans have ever bulit  in there lives and tell us now check out of this really work can you  help me to start ?"
inputs = tokenizer(prompt , return_tensors="pt").to(device)
print(inputs["input_ids"].shape)
print(inputs["attention_mask"].shape)

output = model(**inputs, use_cache=True)

torch.Size([1, 36])
torch.Size([1, 36])


In [ ]:
# checking the next token
print(output.logits.shape)
last_token_logits = output.logits[:,-1,:]
print(last_token_logits.shape)
next_token_id = torch.softmax(last_token_logits,dim=-1)
print(next_token_id.shape)

In [ ]:
# KV key_values stored in it.
past_key_values = output.past_key_values
k , v = output.past_key_values[0]
print(k.shape)
print(v.shape)
print(k.dtype,v.dtype)

size = 1 * 1 * 36 * 256 * 4
print(size/1000, "KB")
# Batch_size, n_head , seq_len , dim

In [ ]:
#  next word predictions.
logits_val , logis_idx = torch.topk(next_token_id,10,dim=-1)

print(logits_val.shape)
print(logis_idx)
top_k_probs = logits_val / torch.sum(logits_val,dim=-1)
print(top_k_probs.shape)
probs = torch.multinomial(top_k_probs,1)
print(probs.shape)
print(probs)
next_token_id = torch.gather(logis_idx, dim=-1, index=probs)
print(next_token_id)

# last token is 
word = tokenizer.decode(next_token_id[0])
print(word)

In [ ]:
# he new k v values get attached to this KV cahce ) past_key_values and we get the logits of the output_next .. 
output_next = model(input_ids=next_token_id,past_key_values=past_key_values)